In [106]:
import os # for handling file operations
from Tkinter import*
root = Tk()
root.title("FootPrint Editor")
root.geometry("1000x600")

#define three functions to handle
def highlight_line(interval=100):
    content_text.tag_remove("active_line", 1.0, "end")
    content_text.tag_add("active_line", "insert linestart", "insert lineend+1c")
    content_text.after(interval, toggle_highlight)

def undo_highlight():
    content_text.tag_remove("active_line", 1.0, "end")

def toggle_highlight(event=None):
    if to_highlight_line.get():
        highlight_line()
    else:
        undo_highlight()
        
        
#displaying line number
def get_line_numbers():
    output = ''
    if show_line_number.get():
        row, col = content_text.index("end").split('.')
        for i in range(1, int(row)):
            output +=  str(i)+ '\n'
    return output

# updating number
def update_line_numbers(event = None):
    line_numbers = get_line_numbers()
    line_number_bar.config(state='normal')
    line_number_bar.delete('1.0', 'end')
    line_number_bar.insert('1.0', line_numbers)
    line_number_bar.config(state='disabled')
    
#theme fxn(changing color theme)
def change_theme(event=None):
    selected_theme = theme_choice.get()
    fg_bg_colors = color_schemes.get(selected_theme)
    foreground_color, background_color = fg_bg_colors.split('.')
    content_text.config(background=background_color, fg=foreground_color)    

    
def on_content_changed(event=None):
    update_line_numbers()
    
    
def cut():
    content_text.event_generate("<<Cut>>")
    return "break"

def copy():
    content_text.event_generate("<<Copy>>")
    return "break"
    
def paste():
    content_text.event_generate("<<Paste>>")
    return "break"

def undo():
    content_text.event_generate("<<Undo>>")
    return "break"

def redo(event=None):
    content_text.event_generate("<<Redo>>")
    return "break"
    
def select_all(event=None):
    content_text.tag_add('sel','1.0','end')
    return "break"

def exit():
    content_text.event_generate("<<Exit>>")
    return "break"

#finding text
def find_text(event=None):
    search_toplevel = Toplevel(root)
    search_toplevel.title('Find Text')
    search_toplevel.transient(root)
    search_toplevel.resizable(False, False)
    Label(search_toplevel, text="Find All:").grid(row=0, column=0, sticky='e')
    search_entry_widget = Entry(search_toplevel, width=25)
    search_entry_widget.grid(row=0, column=1, padx=2, pady=2, sticky='we')
    search_entry_widget.focus_set()
    ignore_case_value = IntVar()
    Checkbutton(search_toplevel, text='Ignore Case',variable=ignore_case_value).grid(row=1, column=1, sticky='e', padx=2, pady=2)
    Button(search_toplevel, text="Find All", underline=0, 
           command=lambda: search_output( search_entry_widget.get(), 
            ignore_case_value.get(), content_text, search_toplevel, 
        search_entry_widget)).grid(row=0, column=2, sticky='e' + 'w', padx=2, pady=2)

    def close_search_window():
        content_text.tag_remove('match', '1.0', END)
        search_toplevel.destroy()
    search_toplevel.protocol('WM_DELETE_WINDOW', close_search_window)
    return "break"    

#search output fxn
def search_output(needle, if_ignore_case, content_text, search_toplevel, search_box):
    content_text.tag_remove('match', '1.0', END)
    matches_found = 0
    if needle:
        start_pos = '1.0'
        while True:
            start_pos = content_text.search(needle, start_pos, nocase=if_ignore_case, stopindex=END)
            if not start_pos:
                break
            end_pos = '{}+{}c'.format(start_pos, len(needle))
            content_text.tag_add('match', start_pos, end_pos)
            matches_found += 1
            start_pos = end_pos
        content_text.tag_config( 'match', foreground='red', background='yellow')
    search_box.focus_set()
    search_toplevel.title('{} matches found'.format(matches_found))

#cursor fxn  info
def on_content_changed(event=None):
    update_line_numbers()
    update_cursor_info_bar()

def show_cursor_info_bar():
    show_cursor_info_checked = show_cursor_info.get()
    if show_cursor_info_checked:
        cursor_info_bar.pack(expand='no', fill=None, side='right', anchor='se')
    else:
        cursor_info_bar.pack_forget()

def update_cursor_info_bar(event=None):
    row, col = content_text.index(INSERT).split('.')
    line_num, col_num = str(int(row)), str(int(col)+1) # col starts at 0 
    infotext = "Line: {0} | Column: {1}".format(line_num, col_num)
    cursor_info_bar.config(text=infotext)
    
    
# Adding Menubar in the widget
menu_bar = Menu(root)
file_menu = Menu(menu_bar, tearoff=0)
menu_bar.add_cascade(label='File', menu=file_menu)
#adding file menu commands
file_menu.add_command(label="New", accelerator='Ctrl+N', compound='left',  underline=0)
file_menu.add_command(label="Open", accelerator='Ctrl+O', compound='left',  underline=0) 
file_menu.add_command(label="Save", accelerator='Ctrl+S', compound='left',  underline=0)
file_menu.add_command(label="Save as", accelerator='Ctrl+Shift+S', compound='left')
file_menu.add_separator()
file_menu.add_command(label="Exit", accelerator='Alt+F4', compound='left',  underline=0,
                      command=exit)


edit_menu = Menu(menu_bar,tearoff=0)
menu_bar.add_cascade(label = 'Edit', menu = edit_menu)
#adding edit menu commands
edit_menu.add_command(label="Undo", accelerator='Ctrl + Z', compound='left',command = undo)
edit_menu.add_command(label="Redo", accelerator='Ctrl + Y', compound='left',command = redo)
edit_menu.add_command(label="Cut", accelerator='Ctrl + X', compound='left', command=cut)
edit_menu.add_separator()
edit_menu.add_command(label="Copy", accelerator='Ctrl + C', compound='left',command=copy)
edit_menu.add_command(label="Paste", accelerator='Ctrl + V', compound='left',command=paste)
edit_menu.add_separator()
edit_menu.add_command(label="Find", accelerator='Ctrl + F', compound='left',
                      underline=1,command=find_text)
edit_menu.add_separator()
edit_menu.add_command(label="Select_All", accelerator='Ctrl + A', compound='left', 
                      underline=7, command = select_all)

view_menu = Menu(menu_bar,tearoff=0)
menu_bar.add_cascade(label = 'View', menu = view_menu)
#adding view menu bar
view_menu.add_command(label=".Normal", compound = 'left')
view_menu.add_command(label="Toolbars", compound = 'left')
view_menu.add_command(label="Status bar", compound = 'left')
view_menu.add_command(label="Styles", accelerator = 'F11', compound = 'left')
view_menu.add_command(label="Zoom", compound = 'left')
#Implementing the View menu
show_line_number = IntVar()
show_line_number.set(1)
view_menu.add_checkbutton(label="Show Line Number", variable = show_line_number,
                         command = update_line_numbers)

show_cursor_info = IntVar()
show_cursor_info.set(1)
view_menu.add_checkbutton(label = "Show cursor location in button",
                          variable = show_cursor_info,command=show_cursor_info_bar)

highlight_line = IntVar()
view_menu.add_checkbutton(label = "Highlight Current Line",onvalue=1,offvalue=0, 
                          variable = highlight_line,command=toggle_highlight)

themes_menu = Menu(menu_bar, tearoff=0)
view_menu.add_cascade(label="Themes", menu = themes_menu)

color_schemes = { 
   'Default': '#000000.#FFFFFF',
   'Greygarious':'#83406A.#D1D4D1',
   'Aquamarine': '#5B8340.#D1E7E0',
   'Bold Beige': '#4B4620.#FFF0E1',
   'Cobalt Blue':'#ffffBB.#3333aa',
   'Olive Green': '#D1E7E0.#5B8340',
   'Night Mode': '#FFFFFF.#000000',
}

theme_choice = StringVar()
theme_choice.set('Default')
for k in sorted(color_schemes):
    themes_menu.add_radiobutton(label=k, variable = theme_choice,command=change_theme)
    

about_menu = Menu(menu_bar,tearoff=0)
menu_bar.add_cascade(label = 'About', menu = about_menu)
#adding about menu bar
about_menu.add_command(label="About", command='left')
about_menu.add_command(label="Help", command='left')

root.config(menu = menu_bar)

#Frame widget to hold icons
shortcut_bar = Frame(root,  height=50, background='light sea green')
shortcut_bar.pack(expand='no', fill='x')
#text widget to hold a line
line_number_bar = Text(root, width=8, padx=3, takefocus=5, border=1,
background='khaki', state='disabled', wrap='none')
line_number_bar.pack(side='left', fill='y')

#scroll bar
content_text = Text(root, wrap='word')
content_text.pack(expand='yes', fill='both')
scroll_bar = Scrollbar(content_text)
content_text.configure(yscrollcommand=scroll_bar.set)
scroll_bar.config(command=content_text.yview)
scroll_bar.pack(side='right', fill='y')

#adding cursor info label
cursor_info_bar = Label(content_text, text='Line: 1 | Column: 1')
cursor_info_bar.pack(expand=NO, fill=None, side=RIGHT, anchor='se')

#handling redo quick
content_text.bind('<Control-A>',select_all)
content_text.bind('<Control-a>',select_all)
content_text.bind('<Control-Y>',redo)
content_text.bind('<Control-y>',undo)
content_text.bind('<Control-f>',find_text)
content_text.bind('<Control-F>',find_text)

content_text.bind('<Any-KeyPress>', on_content_changed)
content_text.tag_configure('active_line', background='ivory2')

root.mainloop()

In [63]:
import Tkinter
help(Tkinter.Text)

Help on class Text in module Tkinter:

class Text(Widget, XView, YView)
 |  Text widget which can display text in various forms.
 |  
 |  Method resolution order:
 |      Text
 |      Widget
 |      BaseWidget
 |      Misc
 |      Pack
 |      Place
 |      Grid
 |      XView
 |      YView
 |  
 |  Methods defined here:
 |  
 |  __init__(self, master=None, cnf={}, **kw)
 |      Construct a text widget with the parent MASTER.
 |      
 |      STANDARD OPTIONS
 |      
 |          background, borderwidth, cursor,
 |          exportselection, font, foreground,
 |          highlightbackground, highlightcolor,
 |          highlightthickness, insertbackground,
 |          insertborderwidth, insertofftime,
 |          insertontime, insertwidth, padx, pady,
 |          relief, selectbackground,
 |          selectborderwidth, selectforeground,
 |          setgrid, takefocus,
 |          xscrollcommand, yscrollcommand,
 |      
 |      WIDGET-SPECIFIC OPTIONS
 |      
 |          autoseparators,

In [65]:
help(Tkinter.Menu)

Help on class Menu in module Tkinter:

class Menu(Widget)
 |  Menu widget which allows displaying menu bars, pull-down menus and pop-up menus.
 |  
 |  Method resolution order:
 |      Menu
 |      Widget
 |      BaseWidget
 |      Misc
 |      Pack
 |      Place
 |      Grid
 |  
 |  Methods defined here:
 |  
 |  __init__(self, master=None, cnf={}, **kw)
 |      Construct menu widget with the parent MASTER.
 |      
 |      Valid resource names: activebackground, activeborderwidth,
 |      activeforeground, background, bd, bg, borderwidth, cursor,
 |      disabledforeground, fg, font, foreground, postcommand, relief,
 |      selectcolor, takefocus, tearoff, tearoffcommand, title, type.
 |  
 |  activate(self, index)
 |      Activate entry at INDEX.
 |  
 |  add(self, itemType, cnf={}, **kw)
 |      Internal function.
 |  
 |  add_cascade(self, cnf={}, **kw)
 |      Add hierarchical menu item.
 |  
 |  add_checkbutton(self, cnf={}, **kw)
 |      Add checkbutton menu item.
 |  
 |  add